In [ ]:
# This allows changes in chakra.py to be automatically re-imported
# (this is tricky with OOP though, to be used with care)
%load_ext autoreload
%autoreload 1
%aimport chakra

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### Simple glacier on BU bed, no calving

In [ ]:
# Get the default params
from oggm import cfg
cfg.initialize()

In [ ]:
# Bassis & Ultee bed profile
fls = chakra.bu_tidewater_bed()
# Constant zero mass-balance
from oggm.core.massbalance import ScalarMassBalance
mb_mod = ScalarMassBalance()

In [ ]:
plt.figure(figsize=(12, 5))
x = fls[-1].dis_on_line * fls[-1].dx_meter / 1000
plt.hlines(0, 0, 60, color='C0')
plt.plot(x, fls[-1].bed_h, color='k')
plt.ylim(-350, 800); plt.xlabel('Distance along flowline [km]'); plt.ylabel('Altitude [m]');

We now instanciate the model. We use a "flux-gate", i.e a constant flux through the left boundary (units: m3 of ice per second). This value can be changed at wish. If the model becomes unstable one can play with the time steps, or the ``flux_gate_buildup`` kwarg. Here are the docs:

In [ ]:
chakra.ChakraModel?

In [ ]:
# The actual model
model = chakra.ChakraModel(fls, mb_model=mb_mod, flux_gate=0.07,
                           fs=5.7e-20*4,  # quite slidy - but this is not slipping - maybe we should add slippping as well
                           )

In [ ]:
# Run for a long time
_, ds = model.run_until_and_store(5000)

In [ ]:
# Mass-conservation check
np.testing.assert_allclose(model.flux_gate_m3_since_y0, ds.volume_m3[-1])
print('Volume: {:.2f} km3'.format(model.flux_gate_m3_since_y0*1e-9))

The diagnostics is a new way to access data that was added to OGGM:

In [ ]:
model.get_diagnostics?

In [ ]:
df_diag = model.get_diagnostics()
df_diag.head()

In [ ]:
plt.figure(figsize=(12, 5))
plt.hlines(0, 0, 60, color='C0')
plt.plot(x, df_diag.bed_h, color='k')
plt.plot(x, df_diag.surface_h, color='C3')
plt.ylim(-350, 800); plt.xlabel('Distance along flowline [km]'); plt.ylabel('Altitude [m]');

### Simple glacier on BU bed, naive calving

I added a mechanism to call an arbitrary function at the end of a standard model time step. I don't know if this will be enough for your purposes (probably not). Another approach would be to modify the code of the class directly, of course. 

Here we illustrate how it works:

In [ ]:
def simple_calving(model, dt):
    """Remove all ice as "calving" when water depth is > 100m.
    
    We will give this func to the model. 
    It will be called at each time step.
   
    It needs to update:
    
    model.calving_m3_since_y0
    model.calving_rate_myr
    model.section

    Parameters
    ----------
    model : the instance of the Chakra model at the moment it
        calls this function
    dt : the chosen timestep according to CFL
    """
    
    # We assume only one flowline (this is OK for now)
    fl = model.fls[-1]

    # Where to remove ice
    loc_remove = np.nonzero(fl.bed_h < -100)[0]
    # How much will we remove
    section = fl.section
    vol_removed = np.sum(section[loc_remove] * fl.dx_meter)
    # Effectively remove mass
    section[loc_remove] = 0

    # Updates so that our parameterization actually does something
    fl.section = section

    # Total calved volume
    model.calving_m3_since_y0 += vol_removed

    # The following is a very silly way to compute calving rate,
    # but the units are ok.

    # Calving rate in units of meter per time
    rate = vol_removed / fl.section[loc_remove[0] - 1]
    # To units of m per year
    model.calving_rate_myr = rate / dt * cfg.SEC_IN_YEAR

    # This is a way for the programmer to add an attribute - here dummy one
    try:
        model.number_of_times_called += 1
    except AttributeError:
        # this happens only the first time
        model.number_of_times_called = 1

We will now pass it to the model and see how it works:

In [ ]:
# Same as before but with the calving param
fls = chakra.bu_tidewater_bed()
mb_mod = ScalarMassBalance()
model = chakra.ChakraModel(fls, mb_model=mb_mod, flux_gate=0.07,
                           fs=5.7e-20*4,  
                           apply_parameterization=simple_calving,  # we just give the func here
                           )

In [ ]:
# Run for a long time
_, ds = model.run_until_and_store(5000)

In [ ]:
# Mass-conservation check (is different)
np.testing.assert_allclose(model.flux_gate_m3_since_y0, ds.volume_m3[-1] + model.calving_m3_since_y0)
print('Volume: {:.2f} km3'.format(ds.volume_m3.data[-1]*1e-9))
print('Calved: {:.2f} km3'.format(model.calving_m3_since_y0*1e-9))

In [ ]:
df_diag_calving = model.get_diagnostics()

In [ ]:
plt.figure(figsize=(12, 5))
plt.hlines(0, 0, 60, color='C0')
plt.plot(x, df_diag.bed_h, color='k')
plt.plot(x, df_diag.surface_h, color='C3', label='Without calving')
plt.plot(x, df_diag_calving.surface_h, color='C2', label='With calving')
plt.ylim(-350, 800); plt.xlabel('Distance along flowline [km]'); plt.ylabel('Altitude [m]');

In [ ]:
# Change units for plot
plt.figure(figsize=(12, 5))
plt.plot(x, df_diag.ice_velocity * cfg.SEC_IN_YEAR, color='C3', label='Without calving')
plt.plot(x, df_diag_calving.ice_velocity * cfg.SEC_IN_YEAR, color='C2', label='With calving')
plt.xlabel('Distance along flowline [km]'); plt.ylabel('Bulk velocity [m yr-1]');

In [ ]:
# Volume evolution with time in the calving case
(ds.volume_m3 * 1e-9).plot();
plt.ylabel('Calving glacier volume [km3]');

In [ ]:
# Calving rate
ds.calving_rate_myr.plot();

That's all for today!

## Water balance calving condition

We define a new condition to enforce stress balance at the terminus.  This will remove ice that fails the plastic terminus balance condition.

In [ ]:
# Define Dimensional and Dimensionless parameters
H0=1e3 #characteristic height for nondimensionalisation 
L0=10e3 #characteristic length (10km)
g = 9.8 #acceleration due to gravity.  Default 9.8 m/s^2
rho_ice = 920.0 #ice density kg/m^3
rho_sea=1020.0 #seawater density kg/m^3

def Bingham_num(tau_y=150e3):
    """Compute the nondimensional Bingham number for this glacier.
    Bingham number can be constant or evolve with basal effective pressure through a Mohr-Coulomb relation.  For now, we use constant.
    
    Parameters
    ----------
    tau_y : yield strength in Pa.  Default 150e3 Pa
    
    Returns
    ----------
    B : the Bingham number
    """
    return tau_y/(rho_ice*g*H0**2/L0)

def BalanceThick(bed, tau_y=150e3):
    """Water balance ice thickness.
    Arguments:
        bed : bed elevation in m a.s.l.
        tau_y : yield strength in Pa.  Default 150e3 Pa

    Returns ice thickness in m for water balance at this point.
    """
    B = Bingham_num(tau_y=tau_y)
    
    bed_nondim = bed/H0
    if bed_nondim<0: #if there is water
        D = -1*bed_nondim
    else:
        D = 0
    balance_thick_nondim = (2*B*H0/L0) + np.sqrt((rho_sea*(D**2)/rho_ice)+(H0*B/L0)**2)
    return H0*balance_thick_nondim
    
def stress_balance_calving(model, dt, yield_strength=150e3):
    """Remove ice seaward of a stress-balanced terminus.  
    We identify the ice thickness of a stress-balanced terminus using the plastic approximation.
    
    We will give this func to the model. 
    It will be called at each time step.
    
    Parameters
    ----------
    model : the instance of the CalvingModel at the moment it
        calls this function
    dt : the chosen timestep according to CFL
    yield_strength : yield strength of glacier ice to feed the model.  Default 150 kPa
    """
    
    fl = model.fls[0]
    
    # Where to remove ice
    ice_thickness = fl.surface_h - fl.bed_h # I think this is going to make an array, right?
    water_present = np.array(fl.bed_h < -10)  # test for water depth too
    balance_thickness = [BalanceThick(b, tau_y=yield_strength) for b in fl.bed_h]
    loc_remove = np.nonzero(ice_thickness < water_present*balance_thickness)[0] # apply both conditions
    # How much will we remove
    section = fl.section
    vol_removed = np.sum(section[loc_remove] * fl.dx_meter)
    # Effectively remove
    section[loc_remove] = 0
    fl.section = section
        
    # Update the model attributes
    # Total calved volume
    model.calving_m3_since_y0 += vol_removed

    # The following is a very silly way to compute calving rate,
    # but the units are ok.

    # Calving rate in units of meter per time
    rate = vol_removed / fl.section[loc_remove[0] - 1]
    # To units of m per year
    model.calving_rate_myr = rate / dt * cfg.SEC_IN_YEAR

Now perform the same test as with simple_calving:

In [ ]:
# Same as before but with stress calving param
fls = chakra.bu_tidewater_bed()
mb_mod = ScalarMassBalance()
model = chakra.ChakraModel(fls, mb_model=mb_mod, flux_gate=0.07,
                           fs=5.7e-20*4,  
                           apply_parameterization=stress_balance_calving,  # we just give the func here
                           )

In [ ]:
# Run for a long time
# Fabi: this is quite slow because of the number of computations
# and possible the strong ice removal creating high velocities and small time steps
# Optimisations might include computing the stress balance only every year, not 
# at each numerical timestep.
_, ds = model.run_until_and_store(5000)

In [ ]:
# Mass-conservation check (is different)
np.testing.assert_allclose(model.flux_gate_m3_since_y0, ds.volume_m3[-1] + model.calving_m3_since_y0)
print('Volume: {:.2f} km3'.format(ds.volume_m3.data[-1]*1e-9))
print('Calved: {:.2f} km3'.format(model.calving_m3_since_y0*1e-9))

In [ ]:
df_diag_calving = model.get_diagnostics()

In [ ]:
plt.figure(figsize=(12, 5))
plt.hlines(0, 0, 60, color='C0')
plt.plot(x, df_diag.bed_h, color='k')
plt.plot(x, df_diag.surface_h, color='C3', label='Without calving')
plt.plot(x, df_diag_calving.surface_h, color='C2', label='With calving')
plt.ylim(-350, 800); plt.xlabel('Distance along flowline [km]'); plt.ylabel('Altitude [m]');

In [ ]:
# Change units for plot
plt.figure(figsize=(12, 5))
plt.plot(x, df_diag.ice_velocity * cfg.SEC_IN_YEAR, color='C3', label='Without calving')
plt.plot(x, df_diag_calving.ice_velocity * cfg.SEC_IN_YEAR, color='C2', label='With calving')
plt.xlabel('Distance along flowline [km]'); plt.ylabel('Bulk velocity [m yr-1]');

In [ ]:
# Volume evolution with time in the calving case
(ds.volume_m3 * 1e-9).plot();
plt.ylabel('Calving glacier volume [km3]');

It looks like our stress-balance condition removed way too much ice.  Let's diagnose: what quantities are we comparing?

We plot what the stress-balance ice thickness is for this yield strength.

In [ ]:
balance_thickness_profile = [BalanceThick(b, tau_y=150e3) for b in model.fls[0].bed_h]
ice_thickness_profile = df_diag.surface_h - model.fls[0].bed_h

plt.figure(figsize=(12,5))
plt.plot(x, balance_thickness_profile)
plt.plot(x, ice_thickness_profile); # compare with the ice thickness in the without-calving case

In [ ]:
# Calving rate
ds.calving_rate_myr.plot();